In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("BankWages_dataset/BankWages.csv", index_col=0)
df.head()

,job,education,gender,minority
1,manage,15,male,no
2,admin,16,male,no
3,admin,12,female,no
4,admin,8,female,no
5,admin,15,male,no


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

categorized_labels = ["job", "gender", "minority"]
for cl in categorized_labels:
    encoded_label = le.fit_transform(df[cl])
    df.drop(cl, axis=1, inplace=True)
    df[cl] = encoded_label
    
df.head()

,education,job,gender,minority
1,15,2,1,0
2,16,0,1,0
3,12,0,0,0
4,8,0,0,0
5,15,0,1,0


In [4]:
df.groupby("minority").describe()

education                                                      job  \
             count       mean       std  min   25%   50%   75%   max  count   
minority                                                                      
0            370.0  13.694595  2.942304  8.0  12.0  15.0  16.0  21.0  370.0   
1            104.0  12.769231  2.555142  8.0  12.0  12.0  15.0  19.0  104.0   

                    ...           gender                                     \
              mean  ...  75%  max  count      mean       std  min  25%  50%   
minority            ...                                                       
0         0.470270  ...  1.0  2.0  370.0  0.524324  0.500084  0.0  0.0  1.0   
1         0.201923  ...  0.0  2.0  104.0  0.615385  0.488860  0.0  0.0  1.0   

                    
          75%  max  
minority            
0         1.0  1.0  
1         1.0  1.0  

[2 rows x 24 columns]

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop("minority", axis=1).values
y = df["minority"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
def identity(x):
    return x

In [7]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [23]:
class Neuron:
    def __init__(self, input_size):
        self.w = np.random.random(input_size)
        self.b = np.random.rand()
    
    def call(self, inputs):
        induced_local_field = np.dot(self.w, inputs) + self.b
        return induced_local_field
    
    def update(self, delta_w, delta_b):
        self.w += delta_w
        self.b += delta_b
        
    @property
    def weights(self):
        return self.w, self.b

In [24]:
class NeuralLayer:
    def __init__(self, inputsize_neuroncount, activation_function):
        self.neurons = [Neuron(inputsize_neuroncount[0]) for _ in range(inputsize_neuroncount[1])]
        self.a_f = activation_function

    def calculate_outputs(self, x_input):
        return [self.a_f(neuron.call(x_input)) for neuron in self.neurons]
    
    @property
    def weights(self):
        return [neuron.weights for neuron in self.neurons]
        

In [38]:
class NNModel:
    def __init__(self, *layers):
        self.layers = layers

    def forward(self, x_input):
        layer_input = x_input
        for layer in self.layers:
            layer_output = layer.calculate_outputs(layer_input)
            layer_input = layer_output

        # output of the neural network is the output of the last layer
        neural_net_output = layer_input
        return neural_net_output

    def fit(self, x_input, y_output):
        pass

    @property
    def weights(self):
        return [neural_layer.weights for neural_layer in self.layers]

In [39]:
n = NNModel(
    NeuralLayer((3,3),identity),
    NeuralLayer((3,3),identity),
)



In [40]:
n.weights

[[(array([0.81237944, 0.50936342, 0.31580488]), 0.22512227494280201),
  (array([0.93930208, 0.1587687 , 0.88381079]), 0.861838162708487),
  (array([0.92970633, 0.47070763, 0.39849654]), 0.1882836518167076)],
 [(array([0.06115986, 0.64434808, 0.26711215]), 0.6992234339936451),
  (array([0.77319595, 0.70855243, 0.29409552]), 0.7200274075304203),
  (array([0.27847557, 0.94376784, 0.34374179]), 0.8849702635184546)]]

In [41]:
# n.fit(X_train[0], y_train[0])
n.forward(X_train[0])

[19.694130235060975, 33.61207555086771, 31.00527605959965]